# Projet RBS structural 

# Introduction 

...
RBPs : qlqs lignes + figure + role dans régulation + role spécifique dans Traduction 

Les reads pour les protéines de liaison à l'ARN (RBP) reflètent le nombre d'interactions entre une RBP spécifique et un ARN particulier associé à chaque gène dans un tableau. Ces données sont généralement obtenues par des techniques comme eCLIP (Enhanced CrossLinking and ImmunoPrecipitation), qui permettent de détecter les sites de liaison en utilisant un cross-linking chimique pour stabiliser l'interaction entre les RBPs et l'ARN, suivi d'une immunoprécipitation pour isoler les complexes RBP-ARN. (generalement dans introduction). 

provenance données : Sypher 
- expliquer exactement comment Sypher a recuperé ces données

Pblm : Trouver les motifs structurales des RBPs qui sont impliqués dans les liaisons à l'ARNm concernant spécifiquement la traduction 

# Data 

## Input Data 

- Pre-processing of the data : 
how he obtained the data 
find a exp ribosome profiling, trouver des cells lines that are same 
data from E-Clip exp -> extended from a data ribosome profile 
RBP df 

Fichier : data_raw.csv 
Le fichier contient  17,670 lignes, ou chaque ligne représente un gène spécifique, et il y a 144 colonnes. 
La première colonne, Geneid, contient les identifiants des gènes.
Les colonnes suivantes représentent des valeurs numériques associées aux RBPs pour chaque gène, données proveant de e-CLIP.
Les quatre dernières colonnes contiennent les données de ribosome profiling d'une meme lignée cellulaire depuis RPF db. 

Une fois isolés, les complexes sont séquencés pour identifier les fragments d'ARN liés à chaque RBP, ce qui permet de quantifier le nombre de reads pour chaque gène. Ces chiffres sont essentiels car ils fournissent des informations sur le rôle des RBPs dans la régulation post-transcriptionnelle.

4 dernières colonnes = conditions expérimentales spécifiques :

- shScramble : Données de contrôle obtenues avec un siRNA "scramble" qui ne cible aucun gène, servant de référence pour évaluer les autres traitements.
- shDDX41 : Données où l'expression du gène DDX41 est inhibée par un shRNA spécifique. Cela permet d'étudier l'impact de cette inhibition sur la traduction des gènes, DDX41 étant impliqué dans la régulation de la traduction et de la maturation des ARN.
- DMSO : Données pour les cellules traitées avec DMSO, un solvant utilisé comme contrôle pour s'assurer que les effets observés ne proviennent pas simplement de la présence du solvant.
- CX5461 : Données pour les cellules traitées avec CX5461, un composé inhibant la synthèse des ARN ribosomiques. Cela permet d'analyser l'effet de ce traitement sur la traduction des gènes en comparaison avec les conditions de contrôle.

In [1]:
import pandas as pd
df = pd.read_csv('data/data_raw.csv')
print(df.head())
print(df.info())

               Geneid  DDX1  XRCC6  GEMIN5  DROSHA  HNRNPUL1  FTO  MORC2  SSB  \
0   ENSG00000186092.7   0.0    0.0     0.0     0.0       0.0  0.0    0.0  0.0   
1   ENSG00000284733.2   0.0    0.0     0.0     0.0       0.0  0.0    0.0  0.0   
2   ENSG00000284662.2   0.0    0.0     0.0     0.0       0.0  0.0    0.0  0.0   
3  ENSG00000187634.13   0.0    0.0     0.0     0.0       0.0  0.0    0.0  0.0   
4  ENSG00000188976.11   0.0    0.0     0.0     0.0       0.0  0.0    0.0  0.0   

   U2AF2  ...  PPIL4  EEF2  SUPV3L1  DDX6      UPF1  TIA1  shScramble  \
0    0.0  ...    0.0   0.0      0.0   0.0  0.000000   0.0    0.000000   
1    0.0  ...    0.0   0.0      0.0   0.0  0.000000   0.0    0.000000   
2    0.0  ...    0.0   0.0      0.0   0.0  0.000000   0.0    0.000000   
3    0.0  ...    0.0   0.0      0.0   0.0  0.000000   0.0    1.146767   
4    0.0  ...    0.0   0.0      0.0   0.0  3.674923   0.0  122.630332   

      shDDX41        DMSO      CX5461  
0    0.000000    0.000000    0.000

## Preparation données 
Vu que les 4 dernieres colonnes correspondent à des conditions experimentales (pas des RBPs directement), on va séparer le data_raw en 2 jeu de données. 

In [2]:
import pandas as pd
df = pd.read_csv("data/data_raw.csv")

df_eclip = df.iloc[:, :-4]
df_rpf = df.iloc[:, [0] + list(range(len(df.columns)-4, len(df.columns)))]  # Garde la première colonne + les 4 dernières

df_eclip.to_csv("df_eclip.csv", index=False, header=True)
df_rpf.to_csv("df_rpf.csv", index=False, header=True)

# Recherche des motifs structuraux des RBPs du data 

## 1- Récupération des Identifiants UniProt
Ecriture d'un script Python qui utilise l'API UniProt pour rechercher les identifiants des protéines RBP en fonction de leurs noms, contenu dans le fichier df_eclip.csv. La fonction get_uniprot_ids a été conçue pour traiter chaque nom de RBP et récupérer son identifiant UniProt correspondant, en veillant à ce que seuls les identifiants associés aux protéines humaines soient retenus. 
A l'issue de cette étape, on recupere les identifants de nos 139 RBPs. 

## 2- Création des URLs d'UniProt
Pour faciliter l'accès aux pages détaillées des RBPs sur le site d'UniProt, nous avons généré des URLs à partir des identifiants obtenus. Ces URLs ont été enregistrées dans un fichier CSV (uniprot_urls.csv)

In [3]:
# execution en 1m36s

import pandas as pd
import requests

def get_uniprot_ids(rbp_names):
    """
    Récupère les identifiants UniProt pour une liste de noms de protéines (RBPs).

    Args:
        rbp_names (list): Une liste de noms de protéines (RBPs) à rechercher.

    Returns:
        dict: Un dictionnaire contenant les identifiants UniProt pour chaque RBP,
              ou un message d'erreur si aucun identifiant n'est trouvé.
    """
    # URL de l'API UniProt pour la recherche
    url = "https://rest.uniprot.org/uniprotkb/search"
    
    # Dictionnaire (noms RBPs comme key et uniprot_id comme value)
    uniprot_ids = {} 

    for rbp in rbp_names:
        params = {
            "query": rbp,  # Recherche par nom de RBP
            "format": "json",   # Format JSON
            "size": 10          # Limite le nombre de résultats à 10
        }
        
        # Envoi de la requête GET à l'API
        response = requests.get(url, params=params)
        
        # Vérification du statut de la requête
        if response.status_code == 200:
            data = response.json()
            # Filtre les résultats pour trouver la protéine humaine
            for result in data.get('results', []):
                if 'HUMAN' in result.get('uniProtkbId', ''):
                    uniprot_ids[rbp] = result['primaryAccession']
                    break  
            else:
                uniprot_ids[rbp] = "Aucun identifiant trouvé pour cette protéine."
        else:
            uniprot_ids[rbp] = f"Erreur: {response.status_code}"
    
    return uniprot_ids


df = pd.read_csv('df_eclip.csv')  
# Extraire les noms des RBPs à partir de la première ligne en excluant la colonne 'geneid'
rbp_names = df.columns.tolist()[1:] 

uniprot_results = get_uniprot_ids(rbp_names)
print(f"{'Nom de la protéine':<20} {'Identifiant UniProt'}")
print('-' * 40)
for rbp, uniprot_id in uniprot_results.items():
    print(f"{rbp:<20} {uniprot_id}")


/Users/abbasi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Nom de la protéine   Identifiant UniProt
----------------------------------------
DDX1                 Q92499
XRCC6                P12956
GEMIN5               Q8TEQ6
DROSHA               Q9NRR4
HNRNPUL1             Q9BUJ2
FTO                  Q9C0B1
MORC2                Q9Y6X9
SSB                  Q04837
U2AF2                P26368
CSTF2T               Q9H0L4
LARP7                Q4G0J3
SLBP                 Q14493
EFTUD2               Q15029
TAF15                Q92804
ELAC2                Q9BQ52
RYBP                 Q8N488
WDR43                Q15061
HNRNPC               P07910
AGGF1                Q8N302
SF3B1                O75533
APOBEC3C             Q9NRW3
GARS                 P41250
RNF187               Q5TA31
SRSF1                Q07955
KHDRBS1              Q07666
YBX3                 P16989
RPS10                P46783
SDAD1                Q9NVU7
HNRNPA1              P09651
ZNF800               Q2TB10
BUD13                Q9BRD0
PRPF8                Q6P2Q9
KHSRP                Q

In [4]:
import csv
base_url = "https://www.uniprot.org/uniprot/"
uniprot_urls = {}

# Enregistrement des URLs dans un fichier CSV
with open("uniprot_urls.csv", "w", newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Nom de la protéine', 'URL UniProt'])  # Header
    for rbp, uniprot_id in uniprot_results.items():
        if "Aucun" not in uniprot_id and "Erreur" not in uniprot_id:
            url = f"{base_url}{uniprot_id}"
            uniprot_urls[rbp] = url
            csv_writer.writerow([rbp, url])
        else:
            csv_writer.writerow([rbp, uniprot_id])

print("\nLes URLs ont été enregistrées dans 'uniprot_urls.csv'.")

# Checking que le fichier est bon 
import pandas as pd
df_urls = pd.read_csv('uniprot_urls.csv')
print(f"Nombre total d'URLs dans le fichier: {len(df_urls)}")
print("\nLes 5 premières lignes du fichier sont :")
print(df_urls.head())


Les URLs ont été enregistrées dans 'uniprot_urls.csv'.
Nombre total d'URLs dans le fichier: 139

Les 5 premières lignes du fichier sont :
  Nom de la protéine                             URL UniProt
0               DDX1  https://www.uniprot.org/uniprot/Q92499
1              XRCC6  https://www.uniprot.org/uniprot/P12956
2             GEMIN5  https://www.uniprot.org/uniprot/Q8TEQ6
3             DROSHA  https://www.uniprot.org/uniprot/Q9NRR4
4           HNRNPUL1  https://www.uniprot.org/uniprot/Q9BUJ2


## 3- Recupération des informations structurales 

L'API UniProt permet de récupérer des informations détaillées sur les protéines, notamment les features qui décrivent les régions fonctionnelles et structurelles. Pour chaque identifiant UniProt, nous avons effectué des requêtes API pour extraire des données au format JSON. Les features comprennent des informations telles que le type, la description, la localisation, l'identifiant et les preuves associées. 
Ces données ont été organisées et stockées dans un fichier CSV (uniprot_structural_info.csv), facilitant ainsi l'analyse et la visualisation des caractéristiques structurales des protéines.

## 4- Tri des informations et récupération des identifiants des BDD 

06/11 update : 
Nous avons repris le fichier précédent pour nettoyer les données et uniquement gardés celles qu'on sera amener à utiliser pour la suite du projet. 

Dans le fichier final on garde les informations structurales essentielles pour chaque protéine RBP. Il inclut une colonne No_feat, qui indique le nombre total de caractéristiques structurales identifiées pour chaque RBP. Les informations clés sur les motifs et les sites de liaison sont regroupées dans une liste, motif_list, qui résume uniquement les caractéristiques pertinentes de type "motif" et "binding site".
Trois autres colonnes, Binding_site, Motifs, et Domains, présentent les identifiants des evidences (bases de données de provenance) à chaque type de caractéristique, séparés par des points-virgules. Chaque identifiant n'apparaît qu'une seule fois pour éviter les doublons. Pour les sites de liaison, l’identifiant est complété par un code d’évidence lorsque nécessaire. 
En fin de compte, le fichier csv fournit un résumé des informations par protéine, avec des colonnes claires pour le nom de la protéine (rbp_name), son identifiant UniProt (uniprotID), le nombre total de caractéristiques (No_feat), et les identifiants des motifs, sites de liaison, et domaines. 

In [5]:
import requests
import pandas as pd

def fetch_structural_info(uniprot_results):
    """
    Récupère les informations structurelles pour une liste d'identifiants UniProt, incluant les motifs, domaines et sites de liaison.

    Pour chaque protéine, cette fonction envoie une requête à l'API UniProt, extrait les caractéristiques structurales et les organise par type.
    Les caractéristiques sont classées en trois catégories : motifs, sites de liaison, et domaines. Une liste détaillée des caractéristiques
    de type "motif" et "binding site" est également créée.
    
    Args:
        uniprot_results (dict): Un dictionnaire contenant les noms de RBPs et leurs identifiants UniProt.

    Returns:
       list: Liste de dictionnaires contenant les informations structurales pour chaque protéine :
            - 'rbp_name', 'uniprotID', 'No_Features', 'motifs', 'binding_Sites', 'domains', et 'motif_list'.
        Utilise des valeurs par défaut en cas d'erreur de récupération.
    """
    
    structural_info = []
    base_url = "https://rest.uniprot.org/uniprotkb/{id}.json"

    for rbp, uniprot_id in uniprot_results.items():
        url = base_url.format(id=uniprot_id)
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])

            binding_site_ids = []
            motif_ids = []
            domain_ids = []
            motif_list = []
            no_features = len(features)
            
            for feature in features:
                feature_type = feature.get('type', 'N/A').lower()
                
                if "binding site" in feature_type or "motif" in feature_type:
                    motif_list.append({
                        'type': feature.get('type', 'N/A'),
                        'description': feature.get('description', 'N/A'),
                        'location': feature.get('location', 'N/A'),
                        'evidences': feature.get('evidences', [])
                    })

                # Extraction des IDs ou evidenceCode pour chaque type : Binding site, Motif, Domain
                if "binding site" in feature_type:
                    for evidence in feature.get('evidences', []):
                        evidence_id = evidence.get('id') or evidence.get('evidenceCode')
                        if evidence_id is not None and evidence_id not in binding_site_ids:
                            binding_site_ids.append(evidence_id)
                
                elif "motif" in feature_type:
                    for evidence in feature.get('evidences', []):
                        evidence_id = evidence.get('id') or evidence.get('evidenceCode')
                        if evidence_id is not None and evidence_id not in motif_ids:
                            motif_ids.append(evidence_id)
                
                elif "domain" in feature_type:
                    for evidence in feature.get('evidences', []):
                        evidence_id = evidence.get('id') or evidence.get('evidenceCode')
                        if evidence_id is not None and evidence_id not in domain_ids:
                            domain_ids.append(evidence_id)

            # Compilation des informations pour chaque RBP dans dico 
            structural_info.append({
                'rbp_name': rbp,
                'uniprotID': uniprot_id,
                'No_Features': no_features,  
                'motifs': ";".join(motif_ids), 
                'binding_Sites': ";".join(binding_site_ids), 
                'domains': ";".join(domain_ids), 
                'motif_list': motif_list 
            })
        else:
            structural_info.append({
                'rbp_name': rbp,
                'uniprotID': uniprot_id,
                'No_Features': 0,  # 0 si échec de récupération
                'motifs': "",
                'binding_Sites': "",
                'domains': "",
                'motif_list': []  # Vide en cas d'erreur
            })

    return structural_info

# Main 
structural_results = fetch_structural_info(uniprot_results)

df_structural = pd.DataFrame([{
    'rbp_name': info['rbp_name'],
    'uniprotID': info['uniprotID'],
    'No_Features': info['No_Features'],
    'motifs': info['motifs'],
    'binding_Sites': info['binding_Sites'],
    'domains': info['domains']
} for info in structural_results])

df_structural.to_csv('RBP_infos.csv', index=False)
print("Informations structurelles sauvegardées dans le fichier 'RBP_infos.csv'.")

Informations structurelles sauvegardées dans le fichier 'RBP_infos.csv'.


After 08/11 

# 5- Design of the Data Structure 

In [6]:
import requests
from collections import namedtuple

# Namedtuple pour chaque caractéristique
FeatureInfo = namedtuple('FeatureInfo', ['Coordinates', 'Description', 'Evidence'])

def fetch_evidence(evidence_list):
    """
    Extrait les preuves d'une caractéristique sous forme de tuples (id, source).

    Args:
        evidence_list (list): Liste des preuves d'une caractéristique.

    Returns:
        list: Liste de tuples contenant (id, source) pour chaque preuve.
    """
    return [(e.get('id', 'N/A'), e.get('source', 'N/A')) for e in evidence_list]

def fetch_structural_info(uniprot_results):
    """
    Récupère les informations structurales pour une liste d'identifiants UniProt,
    organisées dans un dictionnaire de dictionnaires par identifiant UniProt.
    
    Chaque caractéristique est représentée par un namedtuple 'FeatureInfo' avec les champs :
    - Coordinates: (begin, end)
    - Description: Texte de description
    - Evidence: Liste de tuples (id, source)

    Args:
        uniprot_results (dict): Un dictionnaire associant des noms de RBPs à leurs identifiants UniProt.

    Returns:
        dict: Un dictionnaire de dictionnaires contenant les informations structurales pour chaque identifiant UniProt.
    """
    
    structural_info = {}
    base_url = "https://rest.uniprot.org/uniprotkb/{id}.json"

    for rbp, uniprot_id in uniprot_results.items():
        url = base_url.format(id=uniprot_id)
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])

            binding_sites = []
            motifs = []
            domains = []

            for feature in features:
                feature_type = feature.get('type', 'N/A').lower()

                coordinates = (
                    feature.get('location', {}).get('start', {}).get('value', 'N/A'),
                    feature.get('location', {}).get('end', {}).get('value', 'N/A')
                )
                description = feature.get('description', 'N/A')
                
                evidence = fetch_evidence(feature.get('evidences', []))
                
                feature_info = FeatureInfo(Coordinates=coordinates, Description=description, Evidence=evidence)

                if "binding site" in feature_type:
                    binding_sites.append(feature_info)
                elif "motif" in feature_type:
                    motifs.append(feature_info)
                elif "domain" in feature_type:
                    domains.append(feature_info)

            structural_info[uniprot_id] = {
                'motifs': motifs,
                'binding_Sites': binding_sites,
                'domains': domains
            }
        else:
            structural_info[uniprot_id] = {
                'motifs': [],
                'binding_Sites': [],
                'domains': []
            }

    return structural_info

# main 
structural_results = fetch_structural_info(uniprot_results)


A présent structural_results est notre DS sous forme de dictionaraire de dictionnaires avec UniprotID comme clé primaire et les 3  keywords comme clé secondaire. 

## 5.1 Sauvegarde des Informations Structurales avec Indentations pour meilleure lisibilité 

Nous avons enregistré notre structure de données (DS) structural_results dans un fichier texte lisible, en ajoutant des indentations appropriées. structural_results est un dictionnaire de dictionnaires, où chaque identifiant UniProt est la clé primaire, et les trois catégories de caractéristiques (par exemple, "motifs", "binding_Sites", "domains") sont les clés secondaires. Chaque entrée pour une catégorie contient une liste d'objets FeatureInfo avec les champs Coordinates, Description et Evidence.

Pour faciliter la lecture, nous avons utilisé la fonction save_structural_results_text. Cette fonction écrit les données dans un fichier texte, en formatant chaque FeatureInfo pour qu'il ressemble visuellement à sa structure dans le code Python, avec des indentations spécifiques pour chaque niveau de hiérarchie. Ce format facilite la vérification des données structurées pour chaque identifiant UniProt et permet d'assurer que les informations sont correctement organisées et complètes.

In [7]:
def save_structural_results_text(structural_results, filename='structural_results_formatted.txt'):
    with open(filename, 'w') as f:
        for uniprot_id, features in structural_results.items():
            f.write(f"{{'{uniprot_id}': \n")
            f.write("    {\n")

            for feature_type, feature_list in features.items():
                f.write(f"        '{feature_type}': \n")
                f.write("            [\n")
                for feature in feature_list:
                    f.write("             FeatureInfo\n")
                    f.write("             (\n")
                    f.write(f"                 Coordinates={feature.Coordinates},\n")
                    f.write(f"                 Description='{feature.Description}',\n")
                    f.write(f"                 Evidence={feature.Evidence}\n")
                    f.write("             ),\n")
                f.write("            ],\n")

            f.write("    }\n")
            f.write("}\n\n")

    print(f"Les informations structurales ont été sauvegardées dans '{filename}'.")

save_structural_results_text(structural_results)


Les informations structurales ont été sauvegardées dans 'structural_results_formatted.txt'.


In [8]:
# Forme souhaité (exemple avec le fichier json test)

{'Q8N8A6': 
    {'motifs': 
        [FeatureInfo
         (Coordinates=(221, 229), 
          Description='Q motif', 
          Evidence=[]), 
         
         FeatureInfo
         (Coordinates=(371, 374), 
          Description='DEAD box', 
          Evidence=[])], 
    
    'binding_Sites':
        [FeatureInfo
         (Coordinates=(256, 263), 
          Description='', 
          Evidence=[('PRU00541', 'PROSITE-ProRule')])], 
    
    'domains': 
        [FeatureInfo
         (Coordinates=(243, 452), 
          Description='Helicase ATP-binding', 
          Evidence=[('PRU00541', 'PROSITE-ProRule')]), 
         
         FeatureInfo
         (Coordinates=(494, 640), 
          Description='Helicase C-terminal',
          Evidence=[('PRU00542', 'PROSITE-ProRule')])
        ]
    }
}

{'Q8N8A6': {'motifs': [FeatureInfo(Coordinates=(221, 229), Description='Q motif', Evidence=[]),
   FeatureInfo(Coordinates=(371, 374), Description='DEAD box', Evidence=[])],
  'binding_Sites': [FeatureInfo(Coordinates=(256, 263), Description='', Evidence=[('PRU00541', 'PROSITE-ProRule')])],
  'domains': [FeatureInfo(Coordinates=(243, 452), Description='Helicase ATP-binding', Evidence=[('PRU00541', 'PROSITE-ProRule')]),
   FeatureInfo(Coordinates=(494, 640), Description='Helicase C-terminal', Evidence=[('PRU00542', 'PROSITE-ProRule')])]}}